In [1]:
import pandas as pd

import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
# Add project root to sys.path for module imports
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [3]:
from src.utils.data_processing import create_train_and_test_datasets
from src.utils.models import MLP
from src.utils.train_functions import train, evaluate, predict

# Load data

## Load parameter data

In [4]:
# Merge all dataframes
building_parameters = pd.read_csv("/Users/cocoloco/Library/Mobile Documents/com~apple~CloudDocs/Documents/ICAI/4o/AI Lab/data/building_parameters/building_parameters.csv")
building_parameters.set_index("building_id", inplace=True)

In [5]:
building_parameters.head()

,indoor_temp_param,consumption_param,ambient_temp_param,direct_solar_radiation_param,ambient_temp_lag_param
building_id,,,,,
10,0.988916,0.163115,-0.074399,0.000153,0.083117
122,0.955652,1.843034,-0.022571,0.000265,0.067082
129,0.951854,1.661780,0.196741,-0.000472,-0.139513
150,0.975725,0.863512,-0.194279,0.000443,0.214467
183,0.924559,1.000348,-0.118723,0.000107,0.203062


In [6]:
building_parameters.to_csv("/Users/cocoloco/Library/Mobile Documents/com~apple~CloudDocs/Documents/ICAI/4o/AI Lab/data/building_parameters/building_parameters.csv")

## ResStock data

In [7]:
processed_resstock = pd.read_csv("/Users/cocoloco/Library/Mobile Documents/com~apple~CloudDocs/Documents/ICAI/4o/AI Lab/data/resstock/resstock_processed.csv")
processed_resstock.set_index("bldg_id", inplace=True)
processed_resstock.sort_index(inplace=True)

# Create datasets

In [8]:
train_dataset, test_dataset = create_train_and_test_datasets(
    features=processed_resstock,
    labels=building_parameters,
    test_size=0.2,
)

In [9]:
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define model

In [10]:
# HYPERPARAMETERS
# model hyperparameters
input_size = len(processed_resstock.columns)
hidden_sizes = [64, 128, 128]
output_size = len(building_parameters.columns)

# training hyperparameters
num_epochs = 60
learning_rate = 0.0004

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [ ]:
parameter_prediction_model = MLP(
    input_size=input_size,
    hidden_sizes=hidden_sizes,
    output_size=output_size,
).to(device)

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(parameter_prediction_model.parameters(), lr=learning_rate)

# Train model

In [14]:
model_name = "parameter_prediction_model.pth"

In [ ]:
writer = SummaryWriter(log_dir=f"/Users/cocoloco/Library/Mobile Documents/com~apple~CloudDocs/Documents/ICAI/4o/AI Lab/runs/{model_name.replace('.pth', '')}")

In [ ]:
train(
    model=parameter_prediction_model,
    train_loader=train_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device,
    writer=writer,
    verbose=True
)

# Evaluate model

In [ ]:
eval_loss = evaluate(
    model=parameter_prediction_model,
    test_loader=test_loader,
    criterion=criterion,
    device=device,
)

In [ ]:
eval_loss

# Save model

In [12]:
models_path = "/Users/cocoloco/Library/Mobile Documents/com~apple~CloudDocs/Documents/ICAI/4o/AI Lab/models"
if not os.path.exists(models_path):
    os.makedirs(models_path)

In [ ]:
torch.save(parameter_prediction_model.state_dict(), os.path.join(models_path, model_name))
writer.close()

# Load model

In [15]:
model = MLP(
    input_size=input_size,
    hidden_sizes=hidden_sizes,
    output_size=output_size,
).to(device)
model.load_state_dict(torch.load(os.path.join(models_path, model_name)))

<All keys matched successfully>

# Predict

In [20]:
sequences, labels = test_dataset[:16]

In [23]:
predictions = predict(
    model=model,
    sequences=sequences,
    device=device,
)

In [26]:
labels[0]

tensor([ 9.9535e-01,  1.7063e-01,  1.3226e-02,  1.6258e-04, -1.0319e-02])

In [25]:
predictions[0]

tensor([1.0069e+00, 9.4793e-01, 2.5415e-02, 9.0944e-04, 1.4991e-02],
       device='mps:0')